<a href="https://colab.research.google.com/github/leowu0514/Classical-Music-Transformer/blob/master/Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pretty_midi
!gdown https://drive.google.com/uc?id=1E6i0nW5ywc6ETATAUq8rtEVFbSkUEdcz
!gdown https://drive.google.com/uc?id=1gfKauDiiZxW-xSX4jYCVXldfIDb_Y3g0
!gdown https://drive.google.com/uc?id=14SFZ7nAL5OOoYQ9Pap4hyg_seRweuOsV
!gdown https://drive.google.com/uc?id=1maCyJZi2qxwgIwqRL1--0D2bwPwGgmT1


     |████████████████████████████████| 5.6 MB 6.8 MB/s 
     |████████████████████████████████| 51 kB 8.8 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591955 sha256=9b89327ffc5dc1800604ebd9def305c404c08231c29b69010e1fa2ea8483deb5
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi
Downloading...
From: https://drive.google.com/uc?id=1E6i0nW5ywc6ETATAUq8rtEVFbSkUEdcz
To: /content/checkpoint.pt
166MB [00:00, 202MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gfKauDiiZxW-xSX4jYCVXldfIDb_Y3g0
To: /content/util.py
100% 4.89k/4.89k [00:00<00:00, 4.27MB/s]
Downloading...
From: https://drive.google.com/uc?id=14SFZ7nAL5OOoYQ9Pap4hyg_seRweuOsV
To: /content/model.py
100% 8.09k/8.09k [00:00<00:00, 12.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1maCyJZi2qxwgIwqRL1--0D2bwPwGgmT1
To: /content/settings.py
100% 7.62k/7.62k [00:00<00:00, 11.9MB/s]


In [1]:
from model import *
from util import *
from settings import *

In [3]:
### init
d_type = 5
d_barpos = 34
d_tone = 25
d_chord = 109
d_dur = 65
d_pitch = 129
d_attention = 512
heads = 8
N = 12
model = Transformer()
if torch.cuda.is_available():
	model.cuda()

model.load_state_dict(torch.load("./checkpoint.pt"))

<All keys matched successfully>

In [4]:
### make start seq
start_seq = ["bar", "phrase", "pos0", "noteC4dur8"]

### seq to input
type_oh, barpos_oh, tone_oh, chord_oh, dur_oh, pitch_oh = seq2input(start_seq)

In [5]:
### sample how many events
num_event = 200

### sampling
for i in range(len(start_seq) - 1, num_event):
	if i > 127:
		type_input = type_oh[i-127:i+1].unsqueeze(0)
		barpos_input = barpos_oh[i-127:i+1].unsqueeze(0)
		tone_input = tone_oh[i-127:i+1].unsqueeze(0)
		chord_input = chord_oh[i-127:i+1].unsqueeze(0)
		dur_input = dur_oh[i-127:i+1].unsqueeze(0)
		pitch_input = pitch_oh[i-127:i+1].unsqueeze(0)
	else:
		type_input = type_oh[:i+1].unsqueeze(0)
		barpos_input = barpos_oh[:i+1].unsqueeze(0)
		tone_input = tone_oh[:i+1].unsqueeze(0)
		chord_input = chord_oh[:i+1].unsqueeze(0)
		dur_input = dur_oh[:i+1].unsqueeze(0)
		pitch_input = pitch_oh[:i+1].unsqueeze(0)

	with torch.no_grad():
		h, pred_type = model.forward_hidden(type_input, barpos_input, tone_input, chord_input, dur_input, pitch_input, create_mask(type_input))

	pred_type = choose_type(pred_type)
	with torch.no_grad():
		pred_barpos, pred_tone, pred_chord, pred_dur, pred_pitch = model.forward_sampling(h, pred_type)

	type_vec = torch.tensor([0]*d_type).type(torch.FloatTensor).cuda()
	barpos_vec = torch.tensor([0]*d_barpos).type(torch.FloatTensor).cuda()
	tone_vec = torch.tensor([0]*d_tone).type(torch.FloatTensor).cuda()
	chord_vec = torch.tensor([0]*d_chord).type(torch.FloatTensor).cuda()
	dur_vec = torch.tensor([0]*d_dur).type(torch.FloatTensor).cuda()
	pitch_vec = torch.tensor([0]*d_pitch).type(torch.FloatTensor).cuda()

	### Bar & Pos
	if pred_type[0, -1, 0] == 1:
		type_vec[0] = 1

		sampled_barpos = nucleus_sampling(pred_barpos[0, -1, :].softmax(-1).tolist())
		barpos_vec[sampled_barpos] = 1

		tone_vec[0] = 1
		chord_vec[0] = 1
		dur_vec[0] = 1
		pitch_vec[0] = 1

	### tone
	elif pred_type[0, -1, 2] == 1:
		type_vec[2] = 1
		barpos_vec[0] = 1

		sampledtone = nucleus_sampling(pred_tone[0, -1, 1:].softmax(-1).tolist())
		curtone = sampledtone
		tone_vec[sampledtone + 1] = 1
		chord_vec[0] = 1
		dur_vec[0] = 1
		pitch_vec[0] = 1

	### chord
	elif pred_type[0, -1, 3] == 1:
		type_vec[3] = 1
		barpos_vec[0] = 1
		tone_vec[0] = 1

		sampledchord = nucleus_sampling(pred_chord[0, -1, 1:].softmax(-1).tolist())
		curchord = sampledchord
		chord_vec[sampledchord + 1] = 1
		dur_vec[0] = 1
		pitch_vec[0] = 1

	### phrase
	elif pred_type[0, -1, 4] == 1:
		type_vec[4] = 1
		barpos_vec[0] = 1
		tone_vec[0] = 1
		chord_vec[0] = 1
		dur_vec[0] = 1
		pitch_vec[0] = 1

	### notes
	else:
		type_vec[1] = 1
		barpos_vec[0] = 1
		tone_vec[0] = 1
		chord_vec[0] = 1
		sampleddur = nucleus_sampling(pred_dur[0, -1, 1:].softmax(-1).tolist())
		dur_vec[sampleddur + 1] = 1
		sampledpitch = nucleus_sampling(pred_pitch[0, -1, 1:].softmax(-1).tolist())
		pitch_vec[sampledpitch + 1] = 1

	type_oh = torch.cat([type_oh, type_vec.unsqueeze(0)], 0)
	barpos_oh = torch.cat([barpos_oh, barpos_vec.unsqueeze(0)], 0)
	tone_oh = torch.cat([tone_oh, tone_vec.unsqueeze(0)], 0)
	chord_oh = torch.cat([chord_oh, chord_vec.unsqueeze(0)], 0)
	dur_oh = torch.cat([dur_oh, dur_vec.unsqueeze(0)], 0)
	pitch_oh = torch.cat([pitch_oh, pitch_vec.unsqueeze(0)], 0)

In [6]:
### write to midi
item2midi(onehot2item(type_oh, barpos_oh, tone_oh, chord_oh, dur_oh, pitch_oh), "output.mid")